# PaddlePaddle Yolo with OpenVINO - Object Detection Sample Code

This sample shows how to run the PaddlePaddle Yolo Detection algorithm with OpenVINO natively.

PPYOLO Model Configuration: 
- https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.1/configs/ppyolo

PPYOLO Annotation Data:
- https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.2/docs/tutorials/config_annotation/ppyolo_r50vd_dcn_1x_coco_annotation.md


In [1]:
!pip install Pillow
!pip install pyyaml

In [2]:
import os, sys
import numpy as np
import cv2
from openvino.inference_engine import IENetwork, IECore, ExecutableNetwork
from IPython import display
from PIL import Image, ImageDraw
import urllib, shutil, json
import yaml
from yaml.loader import SafeLoader

In [3]:
#Helper functions
def image_preprocess(input_image, size):
    img = cv2.resize(input_image, (size,size))
    img = np.transpose(img, [2,0,1]) / 255
    img = np.expand_dims(img, 0)
    ##NormalizeImage: {mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225], is_scale: True}
    img_mean = np.array([0.485, 0.456,0.406]).reshape((3,1,1))
    img_std = np.array([0.229, 0.224, 0.225]).reshape((3,1,1))
    img -= img_mean
    img /= img_std
    return img.astype(np.float32)

def draw_box(img, results, label_text, scale_x, scale_y):
    for i in range(len(results)):
        #print(results[i])
        bbox = results[i, 2:]
        label_id = int(results[i, 0])
        score = results[i, 1]
        if(score>0.20):
            xmin, ymin, xmax, ymax = [int(bbox[0]*scale_x), int(bbox[1]*scale_y), 
                                      int(bbox[2]*scale_x), int(bbox[3]*scale_y)]
            cv2.rectangle(img,(xmin, ymin),(xmax, ymax),(0,255,0),3)
            font = cv2.FONT_HERSHEY_SIMPLEX
            label_text = label_list[label_id];
            cv2.rectangle(img, (xmin, ymin), (xmax, ymin-70), (0,255,0), -1)
            cv2.putText(img, "#"+label_text,(xmin,ymin-10), font, 1.2,(255,255,255), 2,cv2.LINE_AA)
            cv2.putText(img, str(score),(xmin,ymin-40), font, 0.8,(255,255,255), 2,cv2.LINE_AA)
    return img

In [4]:
!pip install --upgrade paddlepaddle>=2.1 -i https://mirror.baidu.com/pypi/simple

Export static pdmodel from PaddleDetection. It may take a while for the first time to download it. You don't need to compile or install it.  

Please refer to this guide on how to export static model -
https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.2/deploy

Please refer to for PPYolo models -
https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.2/configs/ppyolo/README_cn.md

In [5]:
!git clone git@github.com:PaddlePaddle/PaddleDetection.git

fatal: 目标路径 'PaddleDetection' 已经存在，并且不是一个空目录。


In [6]:
YML_CONFIG="configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml"
PRETRAINED="ppyolo_r50vd_dcn_1x_coco.pdparams"
OUTPUT_DIR="models"

!python PaddleDetection/tools/export_model.py -c PaddleDetection/$YML_CONFIG -o use_gpu=false weights=https://paddledet.bj.bcebos.com/models/$PRETRAINED  TestReader.inputs_def.image_shape=[3,608,608] --output_dir=$OUTPUT_DIR

[08/19 08:19:21] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/ling/.cache/paddle/weights/ppyolo_r50vd_dcn_1x_coco.pdparams
[08/19 08:19:21] ppdet.engine INFO: Export inference config file to models/ppyolo_r50vd_dcn_1x_coco/infer_cfg.yml
[08/19 08:19:26] ppdet.engine INFO: Export model and saved in models/ppyolo_r50vd_dcn_1x_coco


In [7]:
YML_CONFIG="configs/yolov3/yolov3_darknet53_270e_coco.yml"
PRETRAINED="yolov3_darknet53_270e_coco.pdparams"
OUTPUT_DIR="models"

!python PaddleDetection/tools/export_model.py -c PaddleDetection/$YML_CONFIG -o use_gpu=false weights=https://paddledet.bj.bcebos.com/models/$PRETRAINED  TestReader.inputs_def.image_shape=[3,608,608] --output_dir=$OUTPUT_DIR

[08/19 08:19:32] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/ling/.cache/paddle/weights/yolov3_darknet53_270e_coco.pdparams
[08/19 08:19:32] ppdet.engine INFO: Export inference config file to models/yolov3_darknet53_270e_coco/infer_cfg.yml
[08/19 08:19:37] ppdet.engine INFO: Export model and saved in models/yolov3_darknet53_270e_coco


In [8]:
# You can switch between two different models here
#PPYolo
pdmodel_path = "models/ppyolo_r50vd_dcn_1x_coco"
#yolov3
#pdmodel_path = "models/yolov3_darknet53_270e_coco"

pdmodel_file = pdmodel_path + "/model.pdmodel"
pdmodel_config = pdmodel_path + "/infer_cfg.yml"
device = 'CPU'

#load the data from config, and setup the parameters
label_list=[]
with open(pdmodel_config) as f:
    data = yaml.load(f, Loader=SafeLoader)
label_list = data['label_list'];

In [9]:
ie = IECore()
net = ie.read_network(pdmodel_file)

net.reshape({'image': [1, 3, 608, 608], 'im_shape': [
            1, 2], 'scale_factor': [1, 2]})

exec_net = ie.load_network(net, device)
assert isinstance(exec_net, ExecutableNetwork)

In [ ]:
input_image = cv2.imread("horse.jpg")
test_image = image_preprocess(input_image, 608)
test_im_shape = np.array([[608, 608]]).astype('float32')
test_scale_factor = np.array([[1, 2]]).astype('float32')
#print(test_image.shape)

inputs_dict = {'image': test_image, "im_shape": test_im_shape,
               "scale_factor": test_scale_factor}

output = exec_net.infer(inputs_dict)
result_ie = list(output.values())

In [ ]:
result_image = cv2.imread("horse.jpg")
scale_x = result_image.shape[1]/608*2
scale_y = result_image.shape[0]/608
result_image = draw_box(result_image, result_ie[0], label_list, scale_x, scale_y)
_,ret_array = cv2.imencode('.jpg', result_image) 
i = display.Image(data=ret_array)            
display.display(i)

cv2.imwrite("yolo-output.png",result_image) 

In [ ]:
import paddle

def paddle_infer(pdmodel_file, input_dict):
    paddle.enable_static()

    # load model
    exe = paddle.static.Executor(paddle.CPUPlace())
    [inference_program, feed_target_names, fetch_targets] = paddle.static.load_inference_model(
                        os.path.splitext(pdmodel_file)[0], # pdmodel prefix
                        exe)
    
    print(feed_target_names, fetch_targets)

    # run
    output_lod = exe.run(inference_program, feed=input_dict, fetch_list=fetch_targets, return_numpy=False)
    return output_lod

output_lod = paddle_infer(pdmodel_file, inputs_dict)
print(np.array(output_lod[0]).shape, np.array(output_lod[1]).shape)


In [ ]:
result_image = cv2.imread("horse.jpg")
scale_x = result_image.shape[1]/608*2
scale_y = result_image.shape[0]/608
result_image = draw_box(result_image, np.array(output_lod[0]), label_list, scale_x, scale_y)
_,ret_array = cv2.imencode('.jpg', result_image) 
i = display.Image(data=ret_array)            
display.display(i)

cv2.imwrite("yolo-output.png",result_image) 

In [ ]:
def YoloVideo(VideoIndex=0, scale=0.5):
    #PPYolo3
    pdmodel_path = "models/yolov3_darknet53_270e_coco"

    pdmodel_file = pdmodel_path + "/model.pdmodel"
    pdmodel_config = pdmodel_path + "/infer_cfg.yml"
    device = 'CPU'

    #load the data from config, and setup the parameters
    label_list=[]
    with open(pdmodel_config) as f:
        data = yaml.load(f, Loader=SafeLoader)
    label_list = data['label_list'];
    
    ie = IECore()
    net = ie.read_network(pdmodel_file)

    net.reshape({'image': [1, 3, 608, 608], 'im_shape': [
                1, 2], 'scale_factor': [1, 2]})

    exec_net = ie.load_network(net, device)
    assert isinstance(exec_net, ExecutableNetwork)
    
    try:
        cap = cv2.VideoCapture(VideoIndex)
    except:
        print("Cannot Open Device")
        del exec_net
    try:
        ret, frame = cap.read()
        
        while(ret==True):
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            if not ret:
                # Release the Video Device if ret is false
                cap.release()
                # Message to be displayed after releasing the device
                print ("Released Video Resource")
                break
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            half_frame = cv2.resize(frame, (0, 0), fx = scale, fy = scale)
             
            #Processing the Frame
            test_image = image_preprocess(half_frame, 608)
            test_im_shape = np.array([[608, 608]]).astype('float32')
            test_scale_factor = np.array([[1, 2]]).astype('float32')
            #print(test_image.shape)
            inputs_dict = {'image': test_image, "im_shape": test_im_shape,
                           "scale_factor": test_scale_factor}

            output = exec_net.infer(inputs_dict)
            result_ie = list(output.values())
            
            result_image = half_frame.copy()
            #result_image = cv2.resize(result_image, (int(608*1.0),int(608*1.0)))
            scale_x = result_image.shape[1]/608*2
            scale_y = result_image.shape[0]/608
            
            result_image = draw_box(result_image, result_ie[0], label_list, scale_x, scale_y)
            
            #convert to jpg for performance results
            _,ret_array = cv2.imencode('.jpg', result_image) 
            i = display.Image(data=ret_array)            
            
            display.display(i)
            display.clear_output(wait=True)
    except KeyboardInterrupt:
        # Release the Video Device
        cap.release()
        # Message to be displayed after releasing the device
        print("Released Video Resource from KeyboardInterrupt")
        del exec_net
    pass

In [ ]:
YoloVideo(1, 0.5)